# Using our Modified Ray-Tracer
Here we demonstrate how to use our modified ray-tracer (MRT).  For more in-depth discussion of the MRT's inner workings, see our extensive report.

**We want to emphasize that the point of our MRT is the *modifications*, not the underlying ray-tracer.**  Why?  Because the underlying ray-tracer is quite simple, and doesn't have any fancy bells and whistles (e.g., textures and reflection) that you can find in other ray-tracers.  The underlying ray-tracer is also inefficient.  (Case-and-point, it's written in Python rather than, say, Julia or C++.)  **We used the underlying ray-tracer merely as a vehicle to explore and test our modifications.**

That being said, we'll now demonstrate how our MRT can be used.  **See our report for more information, including:** (1) references for more in-depth descriptions and discussions of ray-tracing itself and (2) in-depth discussion of the modifications of our MRT, as well as (4) discussion of performance and (5) suggestions for future improvements and implementation within much more advanced ray-tracers.

### Contents
1. Setup
2. Build a Scene
3. The Standard Ray-Tracer (SRT)
4. The Modified Ray-Tracer (MRT)

## 1: Setup

First let's import our MRT methods, along with some other helpful Python modules.

In [ ]:
##Import necessary modules.
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm as logger
import func_MRT as mrt #The MRT methods
import time as timer
pi = np.pi
plt.close()

##Set booleans.
dosave = False #If True, will save rather than display all verbose plot outputs later on

## 2: Build a Scene

Next, let's build the scene we want to render.  Our underlying ray-tracer has support for adding spheres and planes with non-reflective surfaces, as well as point light sources.  This selection was sufficient for our modification tests.  If you like, you could certainly go and add more complex lights and objects yourself (e.g., cubes and reflective surfaces) to the underlying code ("func_MRT.py").  The underlying class-based ray-tracer is "roomy", in the sense that it was built for possible expansion later on.  Some included functionality (such as paths for the light sources) are not used here, but were put in specifically for later expansions of the code.

In [ ]:
##Generate the worlds and scenes for this test.
eps = 1E-5 #This should be a small number; meant for recursing reflection/shadows for objects

#Generate some lights
light1 = mrt.Light3D_Point(valdict={"coord":np.array([10.0, 10.0, 10.0]), #The x,y,z location of the light source
                                    "scaler":80.0, #A brightness scaler; increasing/decreasing this will brighten/dim the light
                                    "color_norm":1.0, #Leave this as 1.0! Only necessary for more complex underlying ray-tracer
                                    "path":None, #Leave this as 1.0 for point-lights!
                                    "iseverywhere":False} #Leave this as False for point-lights!
                          ) #A first light source
light2 = mrt.Light3D_Point(valdict={"coord":np.array([0.0, 10.0, 2.0]), "scaler":70.0,
                                "color_norm":1.0, "path":None, "iseverywhere":False}) #A second light source; same inputs needed
light3 = mrt.Light3D_Point(valdict={"coord":np.array([-8.0, -8.0, 5.0]), "scaler":65.0,
                                "color_norm":1.0, "path":None, "iseverywhere":False}) #A third light source; same inputs needed
light4 = mrt.Light3D_Point(valdict={"coord":np.array([10.0, 0.0, -2.0]), "scaler":65.0,
                                "color_norm":1.0, "path":None, "iseverywhere":False}) #A fourth light source; same inputs needed

#Generate some spheres
sphere1 = mrt.Sphere3D(valdict={"coord":np.array([-7.0, -5.0, -5.0]), #The x,y,z center of the sphere
                                "rad":5.5, #Sphere radius
                                "color":np.array([0.16, 0.62, 1.0]), #RGB color coordinates (scaled from 0.0 to 1.0)
                                "eps":eps} #This should be a small number; meant for recursing reflection/shadows
                      ) #A first sphere
sphere2 = mrt.Sphere3D(valdict={"coord":np.array([3.4, -6.0, 3.0]), "rad":2.0,
                                "color":np.array([0.38, 0.02, 1.0]), "eps":eps}) #A second sphere
sphere3 = mrt.Sphere3D(valdict={"coord":np.array([-5.0, 6.0, 6.9]), "rad":2.5,
                                "color":np.array([1.0, 0.72, 0.26]), "eps":eps}) #A third sphere
sphere4 = mrt.Sphere3D(valdict={"coord":np.array([0.0, 0.0, 0.0]), "rad":2.5,
                                "color":np.array([0.8, 0.7, 0.8]), "eps":eps}) #A fourth sphere
sphere5 = mrt.Sphere3D(valdict={"coord":np.array([-10.0, 0.0, 0.0]), "rad":5.5,
                                "color":np.array([1.0, 0.77, 0.68]), "eps":eps}) #A fifth sphere

#Generate some planes
path1raw = np.array([0.2,0.5,1.0]) #Unnormalized "path" (i.e., geometric normal) of the plane
path1 = path1raw/np.linalg.norm(path1raw) #Normalized "path" of the plane
plane1 = mrt.Plane3D(valdict={"coord":np.array([0.0, 0.0, -10.0]), #The x,y,z center of the plane's geoemtric normal
                              "path":path1, #The geometric normal of the plane; should be normalized
                              "color":np.array([0.7, 0.5, 1.0]), #RGB color coordinates (scaled from 0.0 to 1.0)
                              "eps":eps} #This should be a small number; meant for recursing reflection/shadows
                    ) #A first plane
path2raw = np.array([1.0,0.0,1.0])
path2 = path2raw/np.linalg.norm(path2raw)
plane2 = mrt.Plane3D(valdict={"coord":np.array([10.0, 0.0, -20.0]), "path":path2,
                              "color":np.array([0.5, 0.5, 0.5]), "eps":eps}) #A second plane

#Collect items into lists
obj_list = [sphere1, sphere2, sphere3, sphere4, sphere5, plane1, plane2]
light_list = [light1, light2, light3, light4]

#Set scene parameters
pixsize = 0.05 #Size of pixels
hspan = 400 #Number of pixels along x-axis of final image
vspan = 400 #Number of pixels along y-axis of final image

#Generate the 'world'
world = mrt.World(obj_list=obj_list, #List of objects
                  light_list=light_list, #List of lights
                  backcolor=np.array([0,0,0])) #Background, lightless color of the image (black!)

#Generate the 'camera'
czval = -25 #Camera's z-axis location (leave as a negative number, since window coordinates are centered at origin)
camera = mrt.Camera(coord=np.array([0,0,czval]), #The x,y,z coordinates of the camera
                    angles=np.array([0,0,0])) #The viewing angles of the camera (leave as 0,0,0 for simplicity)

#Generate the 'window'
window = mrt.Window(hspan=hspan, #Horizontal x-axis number of pixels
                    vspan=vspan, #Verticle y-axis number of pixels
                    coord=np.array([0,0,0]), #The x,y,z coordinates of the window center (centered at origin for simplicity)
                    pixsize=pixsize) #Size of the individual pixels

#Finally, generate the scene
scene = mrt.Scene(world=world, #Set the world
                  camera=camera, #Set the camera
                  window=window) #Set the window

#Window bounds for graphing later
imxy = np.array([window.coord[0]-(0.5*hspan), window.coord[0]+(0.5*hspan), window.coord[1]-(0.5*vspan),
                 window.coord[1]+(0.5*vspan)])*window.pixsize #For graphing later

## 3: The Standard Ray-Tracer (SRT)

Let's see what the rendered scene *should* look like using our Standard Ray-Tracer (SRT).  This ray-tracer is completely *unmodified*.  Note that this may take a few minutes to completely render.

In [ ]:
##Render the scene with the SRT.
print("Running the SRT...")

#Start the timer
timestart_st = timer.time()

#Generate the SRT
renderer_st = mrt.Renderer_Standard(scene=scene, #Insert the scene
                                    sampler=None, #Unnecessary for the SRT
                                    interper=None #Unnecessary for the SRT
                                   ) #Generate the SRT

#Render the scene
res_st = renderer_st.render_scene(form_shade="matte" #"matte" indicates a non-reflective surface
                                 )["image"] #Run the SRT; ["image"] accesses the final image product

#Stop the timer
timetot_st = timer.time() - timestart_st
print("Finished running the SRT!")

In [ ]:
##Display the SRT results.
print("Time for standard ray-tracer: {0}s.".format(timetot_st))
#Graph the SRT results
fig = plt.figure()
ax0 = fig.add_subplot(111)
plt.imshow(res_st, extent=imxy, origin='lower')
plt.suptitle("SRT Results")
plt.title("{0}x{1} grid. Pixel size={2}.".format(hspan, vspan, pixsize))
ax0.xaxis.set_visible(False)
ax0.yaxis.set_visible(False)
plt.show()

## 4: The Modified Ray-Tracer (MRT)

And now, let's use our Modified Ray-Tracer (MRT) to render the same scene.

In [ ]:
##Render the scene with the MRT.
print("Running the MRT...")
verbose = False #If True, will show snapshots of RBF calculations; will increase the total runtime if True

#Start the timer
timestart_in = timer.time()

#Generate a sampler - used to extract a subset of pixels
frac = 0.1 #Fraction of fully ray-traced pixels to use for MRT
sampler = mrt.Sampler_Fraction(valdict={"form":"grid", #"grid" means the subset will be selected on a fixed grid
                                        "fraction":frac} #Fraction of fully ray-traced pixels to use for MRT
                              ) #Fixed grid of samples

#Generate the "interper" - used to interpolate irradiance of unknown pixels via radial basis function (RBF)
interper = mrt.Interper_RBF(valdict={"verbose":verbose, #If True, will show snapshots of RBF calculations
                                     "form":"linear", #Form of RBF; uses scipy.interpolate.RBF under the hood
                                                      #Linear works best, but you can try others allowed by scipy
                                     "max_inpoints":2000, #Maximum number of non-boundary/cap points to include in RBF
                                    "num_cappoints":10} #Number of brightest/darkest points to use for RBF
                           ) #Set the RBF interpolator - see report for more details

#Generate the MRT
sigma = 1 #Sigma for Gaussian filter
cutoff = 0.05 #Normalized cutoff for Gaussian filter
renderer_in = mrt.Renderer_Interpolation(scene=scene, #Set the scene
                                         sampler=sampler, #Set the sampler
                                         interper=interper, #Set the "interper"
                                         verbose=verbose, #If True, will show snapshots of MRT calculations
                                         dofilter=True, #If True, uses Gaussian filter to smooth regions of assigned ids
                                         sigma=sigma, #Sigma for Gaussian filter
                                         cutoff=cutoff #Normalized cutoff for Gaussian filter
                                        ) #Generate the MRT - see report for more details

#Render the scene with the MRT
res_in = renderer_in.render_scene(form_shade="matte")["image"] #Run the MRT

#Stop the timer
timetot_in = timer.time() - timestart_in
print("Done running the MRT!")

In [ ]:
##Display the MRT results.
print("Time for modified ray-tracer: {0}s.".format(timetot_in))
#Graph the MRT results
fig = plt.figure()
ax0 = fig.add_subplot(111)
plt.imshow(res_in, extent=imxy, origin='lower')
plt.suptitle("MRT Results. {0}x{1} grid. Pix. size={2}.".format(hspan, vspan, pixsize))
plt.title("Sample Frac.={3:.2f}, Sigma={4:.1f}, Cut={5:.2f}."
                    .format(hspan, vspan, pixsize, frac, sigma, cutoff))
ax0.xaxis.set_visible(False)
ax0.yaxis.set_visible(False)
plt.show()

#Graph the error
fig = plt.figure()
ax0 = fig.add_subplot(111)
errgrid = np.sum(np.abs(res_in - res_st), axis=2)
score = errgrid.sum()
plt.imshow(errgrid/3.0, cmap=plt.cm.gnuplot2_r, origin='lower', norm=logger(vmin=1E-3, vmax=1))
plt.colorbar()
plt.title(r"$\sum_c$|Error|")
ax0.xaxis.set_visible(False)
ax0.yaxis.set_visible(False)
plt.show()

The results aren't perfect, but they're not half-bad either - especially considering that we used only 10% of the actual data.  See the report for discussions of performance, future improvements, and expansions.

And feel free to play around with the various scene and MRT parameters!  See what cool scenes you can render in less time. :-)